In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd
import re
from pprint import pprint


# 1. Creating dataframe from meta dataset of products

In [12]:
filepath = '/content/drive/MyDrive/BAJAJ/GNN_datasets/amazon-meta.txt'
text = open(filepath, 'r', encoding = 'utf-8')

In [13]:
Ids = []
ASINs = []
titles =[]
salesranks = []
similars = []
num_categories = []
groups = []
categories = []
reviews = []
reviewinfo = []
lc = 0
for line in text:
    lastcat = False
    lastrev = False
    categoies =[]
    lineelements = line.split(':')
    lineelements = [x.strip() for x in lineelements]
    if 'Id' == lineelements[0]:
        Ids.append(lineelements[1])
    elif 'ASIN' == lineelements[0]:
        ASINs.append(lineelements[1])
    elif 'discontinued product' in lineelements:
        Ids.pop(-1)
        ASINs.pop(-1)
    elif 'title' == lineelements[0]:
        titles.append(''.join(lineelements[1]))
    elif 'group' == lineelements[0]:
        groups.append(lineelements[1])
    elif 'salesrank' == lineelements[0]:
        salesranks.append(lineelements[1])
    elif 'similar' == lineelements[0]:
        lineelements.pop(0)
        element = ' '.join(lineelements)
        similars.append(element)
        
    elif 'categories' == lineelements[0]:
        num_categories.append(lineelements[1])
        lastcat = True
        cats = []
        cnt = 0
        for i in range(int(lineelements[1])):
            l = text.readline()
            cats.append('|'.join(l.split('|')[2:]).strip())
        categories.append(cats)
    elif 'reviews' == lineelements[0]:
        review = ':'.join(lineelements[1:]).strip()
        reviews.append(review)
        sub = review.split()
        subsub = [x.split(':') for x in sub]
        r = []
        for i in range(int(subsub[1][1])):
            l = text.readline()
            r.append(l)
        reviewinfo.append(r)

In [14]:
data = {'ID':Ids, 
        'ASIN':ASINs, 
        'title':titles, 
        'group':groups, 
        'salesrank':salesranks, 
        'similar':similars, 
        'num_categories':num_categories, 
        'categories':categories,
        'review':reviews, 
        'reviewinfo':reviewinfo}

In [15]:
df = pd.DataFrame(data)

In [16]:
df.head()

,ID,ASIN,title,group,salesrank,similar,num_categories,categories,review,reviewinfo
0,1,0827229534,Patterns of Preaching,Book,396585,5 0804215715 156101074X 0687023955 0687074...,2,[Subjects[1000]|Religion & Spirituality[22]|Ch...,total:2 downloaded:2 avg rating:5,[ 2000-7-28 cutomer: A2JW67OY8U6HHK ratin...
1,2,0738700797,Candlemas,Book,168596,5 0738700827 1567184960 1567182836 0738700...,2,[Subjects[1000]|Religion & Spirituality[22]|Ea...,total:12 downloaded:12 avg rating:4.5,[ 2001-12-16 cutomer: A11NCO6YTE4BTJ rati...
2,3,0486287785,World War II Allied Fighter Planes Trading Cards,Book,1270652,0,1,[Subjects[1000]|Home & Garden[48]|Crafts & Hob...,total:1 downloaded:1 avg rating:5,[ 2003-7-10 cutomer: A3IDGASRQAW8B2 ratin...
3,4,0842328327,Life Application Bible Commentary,Book,631289,5 0842328130 0830818138 0842330313 0842328...,5,[Subjects[1000]|Religion & Spirituality[22]|Ch...,total:1 downloaded:1 avg rating:4,[ 2004-8-19 cutomer: A2591BUPXCS705 ratin...
4,5,1577943082,Prayers That Avail Much for Business,Book,455160,5 157794349X 0892749504 1577941829 0892749...,2,[Subjects[1000]|Religion & Spirituality[22]|Ch...,total:0 downloaded:0 avg rating:0,[]


In [24]:
df.to_csv('/content/drive/MyDrive/BAJAJ/GNN_datasets/extracteddata.csv')

In [17]:
!pip install fsspec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 5.1 MB/s 


# 2. Feature Extraction and Analysis

In [18]:
pip install dask[dataframe]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import dask.dataframe as dd

In [20]:
def extractrating(x):
    reviewsplit = x['review'].split()
    y = float(reviewsplit[3].split(':')[1])
    return y

In [21]:
def extractdownloads(x):
    reviewsplit = x['review'].split()
    num = int(reviewsplit[1].split(':')[1])
    return num

In [22]:
def extracttotal(x):
    reviewsplit = x['review'].split()
    num = int(reviewsplit[0].split(':')[1])
    return num

In [25]:
df = dd.read_csv('/content/drive/MyDrive/BAJAJ/GNN_datasets/extracteddata.csv').set_index('Unnamed: 0')

In [26]:
df1 = df.copy()
df1['totalreviews'] = df1.apply(extracttotal, axis =1, meta = ('totalreviews', 'i8'))
df1['downloadedreviews'] = df1.apply(extractdownloads, axis = 1, meta = ('downloadedreviews', 'i8'))
df1['avg_rating'] = df1.apply(extractrating, axis = 1, meta = ('avg_rating', 'i8'))
df1 = df1.drop('review', axis = 1)
df1 = df1.reset_index().drop('Unnamed: 0', axis = 1)
df1.ID = df1.ID.astype('int64')
df1.salesrank = df1.salesrank.astype('int64')
df1.num_categories = df1.num_categories.astype('int64')
df1.head()

,ID,ASIN,title,group,salesrank,similar,num_categories,categories,reviewinfo,totalreviews,downloadedreviews,avg_rating
0,1,0827229534,Patterns of Preaching,Book,396585,5 0804215715 156101074X 0687023955 0687074...,2,['Subjects[1000]|Religion & Spirituality[22]|C...,[' 2000-7-28 cutomer: A2JW67OY8U6HHK rati...,2,2,5.0
1,2,0738700797,Candlemas,Book,168596,5 0738700827 1567184960 1567182836 0738700...,2,['Subjects[1000]|Religion & Spirituality[22]|E...,[' 2001-12-16 cutomer: A11NCO6YTE4BTJ rat...,12,12,4.5
2,3,0486287785,World War II Allied Fighter Planes Trading Cards,Book,1270652,0,1,['Subjects[1000]|Home & Garden[48]|Crafts & Ho...,[' 2003-7-10 cutomer: A3IDGASRQAW8B2 rati...,1,1,5.0
3,4,0842328327,Life Application Bible Commentary,Book,631289,5 0842328130 0830818138 0842330313 0842328...,5,['Subjects[1000]|Religion & Spirituality[22]|C...,[' 2004-8-19 cutomer: A2591BUPXCS705 rati...,1,1,4.0
4,5,1577943082,Prayers That Avail Much for Business,Book,455160,5 157794349X 0892749504 1577941829 0892749...,2,['Subjects[1000]|Religion & Spirituality[22]|C...,[],0,0,0.0


In [27]:
df2 = df1[['ID', 'reviewinfo']].copy()
df2.reviewinfo = df2.reviewinfo.str.strip('][').str.replace("'", '').str.split(',')
df2 = df2.explode('reviewinfo').reset_index()
df2 = df2.drop(['index'], axis = 1)
df1 = df1.drop('reviewinfo', axis = 1)
df2.head()

,ID,reviewinfo
0,1,2000-7-28 cutomer: A2JW67OY8U6HHK rating...
1,1,2003-12-14 cutomer: A2VE83MZF98ITY rati...
2,2,2001-12-16 cutomer: A11NCO6YTE4BTJ ratin...
3,2,2002-1-7 cutomer: A9CQ3PLRNIR83 rating...
4,2,2002-1-24 cutomer: A13SG9ACZ9O5IM ratin...


In [28]:
df3 = df1[['ID', 'categories']].copy()
df3['categories'] = df3.categories.str.strip('][').str.replace("'", '').str.split(',')
df3 = df3.explode('categories').reset_index()
df3 = df3.drop('index', axis = 1)
df1 = df1.drop(['categories'], axis = 1)
df3.head()

,ID,categories
0,1,Subjects[1000]|Religion & Spirituality[22]|Chr...
1,1,Subjects[1000]|Religion & Spirituality[22]|Ch...
2,2,Subjects[1000]|Religion & Spirituality[22]|Ear...
3,2,Subjects[1000]|Religion & Spirituality[22]|Ea...
4,3,Subjects[1000]|Home & Garden[48]|Crafts & Hobb...


In [29]:
df4 = df1[['ID', 'similar']].copy()
df4.similar = df4.similar.str.split()
df4 = df4.reset_index().drop('index', axis =1)
df1 = df1.drop('similar', axis = 1)
df4.head()

,ID,similar
0,1,"[5, 0804215715, 156101074X, 0687023955, 068707..."
1,2,"[5, 0738700827, 1567184960, 1567182836, 073870..."
2,3,[0]
3,4,"[5, 0842328130, 0830818138, 0842330313, 084232..."
4,5,"[5, 157794349X, 0892749504, 1577941829, 089274..."


In [30]:
df1.compute().to_csv('/content/drive/MyDrive/finalpreprocesseddata.csv', index = False)
df2.compute().to_csv('/content/drive/MyDrive/reviews.csv', index = False)
df3.compute().to_csv('/content/drive/MyDrive/categories.csv', index = False)
df4.compute().to_csv('/content/drive/MyDrive/similar.csv', index = False)